In [1]:
import numpy as np

%reload_ext autoreload
%autoreload 2

from MTPHandler import Plate
from MTPHandler.examples import parse_spectramax # custom parser for SpectraMax plate reader

# MTPManager - MTPy?

MTPManager provides an API to work with microtiter plate data. Core features are:
- read in experimental data from a microtiter plate to a `Plate` object model
- assign initial concentrations to of different species (chemicals or proteins) to individual wells
- automatically blank the data of the plate data with respect to initial concentrations of different species
- automatically identify standards of a chemical and make a calibration model for concentration calculation
- automatically identify reaction data and write it to a EnzymeMLDocument

## Read in experimental data
A `Plate` is created by loading the output file of a plate reader. 

<div class="alert alert-block alert-info"><b>Info:</b> Since the output format differs between plate readers, one must provide parsing function which addresses the object model describing a `Plate`. Once, the reader is available, all features of this tool are applicable..</div>

In [2]:
# Path to spectrometer output file
path = "/Users/max/Documents/enzymeml/alaric/EnzymeML_SLAC_5xConcentration_pH2.0-3.5/ABTS_EnzymeML_340nm_420nm_5x_35deg_pH3.5.txt"

# Load plate
plate = Plate.from_reader(
    reader=parse_spectramax,
    path=path,
    time=np.linspace(0, 15, 31),
    time_unit="min",
    ph=3.5,
    temperature=35,
)

### Define Species present in wells of the plate

In order to identify wells with catalyzed reactions and standards for calibration, reactants and proteins must be defined. This is done by calling `add_reactant()` or `add_protein()` to either add a chemical or a protein to the list of species. The function takes the name of the species as argument. Besides the name ans wether or not the concentration of the species is constant throughout the experiment, even more information can be added as keyword arguments. All possible arguments are listed in the [specifications of EnzymeML](https://github.com/EnzymeML/enzymeml-specifications/blob/main/specifications/enzymeml.md#abstractspecies).

In [3]:
# Define species
buffer = plate.add_reactant(id="s1", name="Buffer", constant=True)

abts = plate.add_reactant(id="s0", name="ABTS", constant=False)

abts_radical = plate.add_reactant(id="s2", name="ABTS_radical", constant=False)

slac = plate.add_protein(id="p0", name="SLAC", constant=True, sequence="MSSKSKPKDVKV")

### Assign initial concentrations off different species to individuals wells

Initial concentrations of different species can be assigned to individual wells by calling `assign_species()`. The function can map arrays of initial concentrations `to='rows'` or `to='columns'`. Alternatively, a single concentration can be mapped `to='all'` or `to='except'` wells. 

```python

In [4]:
# Assign buffer concentration
plate.assign_species(to="all", species=buffer, init_conc=100, conc_unit="mmol / l")

# Assign substrate concentrations
plate.assign_species(
    to="rows",
    species=abts,
    init_conc=[0, 5, 10, 15, 25, 50, 75, 100, 150, 200],
    conc_unit="umol / l",
    ids=[],
)

# Assign enzyme concentrations
plate.assign_species(
    to="rows", ids=["A", "B", "C"], species=slac, init_conc=4, conc_unit="umol / l"
)

In this example the rows A-C contain enzyme reactions. For later modeling we introduce the product species to these wells and assign initial concentrations of 0 uM to the rows.

In [5]:
# Assign product concentrations
plate.assign_species(
    to="rows",
    ids=["A", "B", "C"],
    species=abts_radical,
    init_conc=0,
    conc_unit="umol / l",
)

### Blank measurement data

Since we have assigned the initial concentrations for each species, blanking id carried out semi-automatically. By calling ```blank_species()```, the absorption contribution of the defined species is subtracted from all wells, in which the species is present.

__How does it work?__  
After specifying the concentration of each species present in each well for each well respectively, wells which were prepared for blanking are be implicitly identified.
If we call the blank function and specify a species which should be blanked, the blanking function first identifies wells which solely contain the specified species. Thereafter, the mean value across all identified wells for blanking is calculated. Thereafter, all wells which contain the selected species are selected and the calculated blank is subtracted from all absorption values. Lastly, a flag (```was_blanked = True```) is set, that prevents the species from being used for blanking again

In [6]:
# Remove the absorption contribution of buffer and protein
plate.blank_species(species=buffer, wavelength=340)
plate.blank_species(species=slac, wavelength=340)

Blanked 60 wells containing Buffer.
Blanked 30 wells containing SLAC.


### Create `Standard` for concentration calculation

After the absorption data is cleaned a standard can be generated which can then be used for concentration calculation. By calling the `calibrate()` method, a `Calibrator` from [CaliPytion](https://github.com/FAIRChemistry/CaliPytion) is instantiated from the standard data of the plate. Its predefined models are fitted to the standard data by calling the `fit_models()` method.

In [7]:
abts_calibration = plate.calibrate(species=abts, wavelength=340, cutoff=3)

# Get predefined calibration models
linear, quadratic, cubic, rational = abts_calibration.models

# Fit models
abts_calibration.fit_models()

,AIC,R squared,RMSD
Model Name,,,
cubic,-114,0.9919,0.0823
quadratic,-104,0.9864,0.1063
rational,-103,0.9860,0.1079
linear,-98,0.9816,0.1237


In [8]:
# Visualize fitted calibration models
abts_calibration.visualize(cubic)

After visualizing the fitted model with standard data and comparing and assessing statistical parameters of the models, the best model to be used for concentration calculation is added to the standard data by calling the `save_model()` method.

In [9]:
# Save best calibration model to `Standard`
abts_standard = abts_calibration.save_model(cubic)

## Create an `EnzymeMLDocument` with concentration data from the plate

Catalyzed reaction data can be mapped to an `EnzymeMLDocument` by calling the `to_enzymeml()` method. It identifies wells containing proteins and the specified 'detected_reactant'. If a `Standard` with a fitted calibration model is provided, the absorption values are are converted into concentration data. If measured absorption data is outside of calibration bonds of the model for concentration calculation, the respective values are replaced with `float('nan')` values.

In [12]:
enzymeml = plate.to_enzymeml(
    name="SLAC kinetic assay",
    detected_reactant=abts,
    reactant_standard=abts_standard,
    wavelength=340,
    path="SLAC_kinetic_assay.json",
)

Found 27 catalyzed wells


In [11]:
print(enzymeml)

EnzymeMLDocument
├── id = enzymemldocument0
├── name = SLAC kinetic assay
├── created = 2023-09-12 10:38:34
├── vessels
│   └── 0
│       └── Vessel
│           ├── id = plate0
│           ├── name = MTP 96 well
│           ├── volume = 200.0
│           ├── unit = ul
│           └── constant = True
├── proteins
│   └── 0
│       └── Protein
│           ├── id = p0
│           ├── name = SLAC
│           ├── vessel_id = plate0
│           ├── constant = True
│           ├── sequence = MSSKSKPKDVKV
│           └── ontology = SBO:0000013
├── reactants
│   ├── 0
│   │   └── Reactant
│   │       ├── id = s1
│   │       ├── name = Buffer
│   │       ├── vessel_id = plate0
│   │       ├── constant = True
│   │       └── ontology = SBO:0000247
│   ├── 1
│   │   └── Reactant
│   │       ├── id = s0
│   │       ├── name = ABTS
│   │       ├── vessel_id = plate0
│   │       ├── constant = False
│   │       └── ontology = SBO:0000247
│   └── 2
│       └── Reactant
│           ├── id = s2
│       